# Imports

In [1]:
from itertools import product
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings

In [2]:
warnings.filterwarnings('ignore')

# Configs

In [3]:
INPUT_PATH = '/Users/maedeh/Desktop/demand_project/demand_project/shoofer-demand-prediction/data/features_df_phase2.parquet'
Ridge_PATH = '/Users/maedeh/Desktop/demand_project/demand_project/shoofer-demand-prediction/data/Ridge_feature_predictions_phase2.parquet'
RF_ridge_PATH = '/Users/maedeh/Desktop/demand_project/demand_project/shoofer-demand-prediction/data/RF_ridge_predictions_phase2.parquet'
RF_PATH = '/Users/maedeh/Desktop/demand_project/demand_project/shoofer-demand-prediction/data/RF_predictions_phase2.parquet'



FEATURE_LIST = ['zone',
                'day_of_week',
                'day_of_month',

                'group_max',
                'group_min',
                
                'max_previous_week_interval',
                'max_previous_2week_interval',
    
                'max_previous_7exact_interval',
                'max_previous_14exact_interval',
    
                'previous_day_9interval',
                'previous_day_10interval',
                'previous_day_17interval',

                'diff_previous_2day_interval',

                'diff_previous_2day_previous_interval',
    
                'previous_day_interval',
                'previous_2day_interval',
                'previous_3day_interval',
                'previous_4day_interval',
                'previous_5day_interval',
                'previous_6day_interval',
                'previous_week_interval',
                'previous_8day_interval',
                'previous_9day_interval',
                'previous_10day_interval',
                'previous_11day_interval',
                'previous_12day_interval',
                'previous_13day_interval',
                'previous_2week_interval'
               ]


TEST_START_DATE = '2023-04-1'
VALIDATION_START_DATE = '2023-03-18'

NUM_INTERVAL_PER_DAY = 8
HIGH_DEMAND_NUMS = 55
MID_DEMAND_NUMS = 157
sort_method = 'mean'

AUTO_TUNE = True
add_ridge_feature = True

# Data preparation


## Load Data

In [4]:
features_df = pd.read_parquet(INPUT_PATH, engine='pyarrow')

In [5]:
print(f'features dataframe shape : {features_df.shape}')
features_df.head()

features dataframe shape : (251520, 7)


,Location,Date,Hour_interval,Demand,previous_day_interval,previous_week_interval,previous_2week_interval
0,1,2023-01-01,0,0.0,NaN,NaN,NaN
1,1,2023-01-01,3,1.0,NaN,NaN,NaN
2,1,2023-01-01,6,1.0,NaN,NaN,NaN
3,1,2023-01-01,9,1.0,NaN,NaN,NaN
4,1,2023-01-01,12,13.0,NaN,NaN,NaN


In [6]:
ridge_df = pd.read_parquet(Ridge_PATH, engine='pyarrow')

In [7]:
print(f'rides dataframe shape : {ridge_df.shape}')
ridge_df.head()

rides dataframe shape : (222176, 4)


,Location,Date,Hour_interval,Predicted_demand
0,1,2023-01-15,0,-0.046404
1,1,2023-01-15,3,0.610225
2,1,2023-01-15,6,2.055764
3,1,2023-01-15,9,1.146677
4,1,2023-01-15,12,5.979035


## Add feature

In [8]:
def add_feature(dataset, lag_num):
    
    for i in range(1,lag_num):
        if i not in(1,7):
            dataset[f'previous_{i}day_interval'] = dataset.groupby('Location')['Demand'].shift(i*NUM_INTERVAL_PER_DAY)

    
    dataset['day_of_week'] = dataset['Date'].dt.dayofweek
    dataset['day_of_month'] = dataset['Date'].dt.day
    dataset['time'] = dataset['Hour_interval']
    dataset['zone'] = dataset['Location']

    
    dataset['max_previous_week_interval'] = dataset.groupby('Location')['Demand'].rolling(window = 7*NUM_INTERVAL_PER_DAY).max().reset_index(drop=True)
    dataset['max_previous_2week_interval'] = dataset.groupby('Location')['Demand'].rolling(window = 14*NUM_INTERVAL_PER_DAY).max().reset_index(drop=True)

    
    df = dataset.sort_values(['Location','Hour_interval','Date'])[['Location','Hour_interval','Date','Demand']]
    df['max_previous_7exact_interval'] = dataset.groupby(['Location','Hour_interval'])['Demand'].rolling(window = 7).max().values
    dataset['max_previous_7exact_interval'] = df.sort_values(['Location', 'Date','Hour_interval'])['max_previous_7exact_interval']
    df['max_previous_14exact_interval'] = dataset.groupby(['Location','Hour_interval'])['Demand'].rolling(window = 14).max().values
    dataset['max_previous_14exact_interval'] = df.sort_values(['Location', 'Date','Hour_interval'])['max_previous_14exact_interval']

    
    dataset['previous_day_9interval'] = dataset.groupby('Location')['Demand'].shift(NUM_INTERVAL_PER_DAY+1)
    dataset['previous_day_10interval'] = dataset.groupby('Location')['Demand'].shift(NUM_INTERVAL_PER_DAY+2)
    dataset['previous_day_17interval'] = dataset.groupby('Location')['Demand'].shift(NUM_INTERVAL_PER_DAY+17)

    
    dataset['diff_previous_2day_previous_interval'] = dataset['previous_day_17interval']-dataset['previous_day_9interval']

    
    dataset['diff_previous_2day_interval'] = dataset['previous_day_interval']-dataset['previous_2day_interval']
    dataset['diff_previous_2week_interval'] = dataset['previous_week_interval']-dataset['previous_2week_interval']

    
    sort_index = dataset.groupby(['Location','Hour_interval'])['Demand'].aggregate([sort_method]).sort_values(sort_method, ascending=False).reset_index()
    sorted_index = sort_index[['Location', 'Hour_interval']]
    high_demand_df = dataset.merge(sorted_index[:HIGH_DEMAND_NUMS], on=['Location','Hour_interval'])
    mid_demand_df = dataset.merge(sorted_index[HIGH_DEMAND_NUMS:HIGH_DEMAND_NUMS+MID_DEMAND_NUMS], on=['Location','Hour_interval'])
    low_demand_df = dataset.merge(sorted_index[HIGH_DEMAND_NUMS+MID_DEMAND_NUMS:], on=['Location','Hour_interval'])
    high_demand_df['group_max'] = high_demand_df['Demand'].max()
    high_demand_df['group_min'] = high_demand_df['Demand'].min()
    mid_demand_df['group_max'] = mid_demand_df['Demand'].max()
    mid_demand_df['group_min'] = mid_demand_df['Demand'].min()
    low_demand_df['group_max'] = low_demand_df['Demand'].max()
    low_demand_df['group_min'] = low_demand_df['Demand'].min()
    dataset = pd.concat([high_demand_df, mid_demand_df, low_demand_df]).sort_values(['Location','Date','Hour_interval'])
    
    return dataset



In [9]:
new_features_df = add_feature(features_df, lag_num = 14)
print(f'new features dataframe shape : {new_features_df.shape}')
new_features_df.head()

new features dataframe shape : (251520, 34)


,Location,Date,Hour_interval,Demand,previous_day_interval,previous_week_interval,previous_2week_interval,previous_2day_interval,previous_3day_interval,previous_4day_interval,...,max_previous_7exact_interval,max_previous_14exact_interval,previous_day_9interval,previous_day_10interval,previous_day_17interval,diff_previous_2day_previous_interval,diff_previous_2day_interval,diff_previous_2week_interval,group_max,group_min
0,1,2023-01-01,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1030.0,0.0
120,1,2023-01-01,3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1030.0,0.0
240,1,2023-01-01,6,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1030.0,0.0
360,1,2023-01-01,9,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1030.0,0.0
480,1,2023-01-01,12,13.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1030.0,0.0


In [10]:
def feature_selection(dataset, FEATURE_LIST):
    
    dataset = dataset[['Location','Date','Hour_interval','Demand']+FEATURE_LIST]
    dataset.dropna(inplace = True)
    
    return dataset

In [11]:
selected_features_df = feature_selection(new_features_df, FEATURE_LIST)
print(f'features dataframe shape : {selected_features_df.shape}')
selected_features_df.head()

features dataframe shape : (222176, 32)


,Location,Date,Hour_interval,Demand,zone,day_of_week,day_of_month,group_max,group_min,max_previous_week_interval,...,previous_5day_interval,previous_6day_interval,previous_week_interval,previous_8day_interval,previous_9day_interval,previous_10day_interval,previous_11day_interval,previous_12day_interval,previous_13day_interval,previous_2week_interval
14,1,2023-01-15,0,0.0,1,6,15,1030.0,0.0,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
134,1,2023-01-15,3,2.0,1,6,15,1030.0,0.0,14.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,2.0,3.0,1.0
254,1,2023-01-15,6,1.0,1,6,15,1030.0,0.0,14.0,...,0.0,2.0,3.0,1.0,0.0,2.0,2.0,2.0,4.0,1.0
374,1,2023-01-15,9,0.0,1,6,15,1030.0,0.0,14.0,...,1.0,1.0,1.0,1.0,0.0,2.0,0.0,2.0,4.0,1.0
494,1,2023-01-15,12,8.0,1,6,15,1030.0,0.0,14.0,...,1.0,1.0,2.0,1.0,3.0,4.0,3.0,7.0,11.0,13.0


In [12]:
def join_ridge_feature(dataset, ridge_df, FEATURE_LIST):

    dataset = dataset.merge(ridge_df, how='outer', on = ['Location','Date','Hour_interval'])
    FEATURE_LIST.append('Predicted_demand')
    
    return dataset, FEATURE_LIST

In [13]:
if add_ridge_feature:
    selected_features_df, FEATURE_LIST = join_ridge_feature(selected_features_df, ridge_df, FEATURE_LIST)
    print(f'final features dataframe shape : {selected_features_df.shape}')
selected_features_df.head()

final features dataframe shape : (222176, 33)


,Location,Date,Hour_interval,Demand,zone,day_of_week,day_of_month,group_max,group_min,max_previous_week_interval,...,previous_6day_interval,previous_week_interval,previous_8day_interval,previous_9day_interval,previous_10day_interval,previous_11day_interval,previous_12day_interval,previous_13day_interval,previous_2week_interval,Predicted_demand
0,1,2023-01-15,0,0.0,1,6,15,1030.0,0.0,14.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,-0.046404
1,1,2023-01-15,3,2.0,1,6,15,1030.0,0.0,14.0,...,1.0,0.0,0.0,1.0,1.0,0.0,2.0,3.0,1.0,0.610225
2,1,2023-01-15,6,1.0,1,6,15,1030.0,0.0,14.0,...,2.0,3.0,1.0,0.0,2.0,2.0,2.0,4.0,1.0,2.055764
3,1,2023-01-15,9,0.0,1,6,15,1030.0,0.0,14.0,...,1.0,1.0,1.0,0.0,2.0,0.0,2.0,4.0,1.0,1.146677
4,1,2023-01-15,12,8.0,1,6,15,1030.0,0.0,14.0,...,1.0,2.0,1.0,3.0,4.0,3.0,7.0,11.0,13.0,5.979035


## Split Train and Test Data

In [15]:
def train_test_splitting(dataset, START_DATE):

    train_df = dataset[dataset['Date'] < START_DATE]
    test_df = dataset[dataset['Date'] >= START_DATE]

    return train_df, test_df

In [16]:
train_df, test_df = train_test_splitting(selected_features_df, TEST_START_DATE)

In [17]:
print(f'train dataframe shape : {train_df.shape}')
train_df.head()

train dataframe shape : (159296, 33)


,Location,Date,Hour_interval,Demand,zone,day_of_week,day_of_month,group_max,group_min,max_previous_week_interval,...,previous_6day_interval,previous_week_interval,previous_8day_interval,previous_9day_interval,previous_10day_interval,previous_11day_interval,previous_12day_interval,previous_13day_interval,previous_2week_interval,Predicted_demand
0,1,2023-01-15,0,0.0,1,6,15,1030.0,0.0,14.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,-0.046404
1,1,2023-01-15,3,2.0,1,6,15,1030.0,0.0,14.0,...,1.0,0.0,0.0,1.0,1.0,0.0,2.0,3.0,1.0,0.610225
2,1,2023-01-15,6,1.0,1,6,15,1030.0,0.0,14.0,...,2.0,3.0,1.0,0.0,2.0,2.0,2.0,4.0,1.0,2.055764
3,1,2023-01-15,9,0.0,1,6,15,1030.0,0.0,14.0,...,1.0,1.0,1.0,0.0,2.0,0.0,2.0,4.0,1.0,1.146677
4,1,2023-01-15,12,8.0,1,6,15,1030.0,0.0,14.0,...,1.0,2.0,1.0,3.0,4.0,3.0,7.0,11.0,13.0,5.979035


In [18]:
print(f'test dataframe shape : {test_df.shape}')
test_df.head()

test dataframe shape : (62880, 33)


,Location,Date,Hour_interval,Demand,zone,day_of_week,day_of_month,group_max,group_min,max_previous_week_interval,...,previous_6day_interval,previous_week_interval,previous_8day_interval,previous_9day_interval,previous_10day_interval,previous_11day_interval,previous_12day_interval,previous_13day_interval,previous_2week_interval,Predicted_demand
608,1,2023-04-01,0,0.0,1,5,1,1030.0,0.0,6.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.047126
609,1,2023-04-01,3,1.0,1,5,1,1030.0,0.0,6.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,2.0,0.047126
610,1,2023-04-01,6,1.0,1,5,1,1030.0,0.0,6.0,...,0.0,3.0,0.0,5.0,2.0,0.0,1.0,1.0,7.0,0.047126
611,1,2023-04-01,9,0.0,1,5,1,1030.0,0.0,6.0,...,0.0,2.0,1.0,1.0,3.0,1.0,1.0,2.0,2.0,0.047126
612,1,2023-04-01,12,1.0,1,5,1,1030.0,0.0,6.0,...,5.0,3.0,2.0,4.0,0.0,0.0,3.0,9.0,2.0,0.047126


In [19]:
gridsearch_train_df, validation_df = train_test_splitting(train_df, VALIDATION_START_DATE)

In [20]:
print(f'gridsearch_train dataframe shape : {gridsearch_train_df.shape}')
gridsearch_train_df.head()

gridsearch_train dataframe shape : (129952, 33)


,Location,Date,Hour_interval,Demand,zone,day_of_week,day_of_month,group_max,group_min,max_previous_week_interval,...,previous_6day_interval,previous_week_interval,previous_8day_interval,previous_9day_interval,previous_10day_interval,previous_11day_interval,previous_12day_interval,previous_13day_interval,previous_2week_interval,Predicted_demand
0,1,2023-01-15,0,0.0,1,6,15,1030.0,0.0,14.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,-0.046404
1,1,2023-01-15,3,2.0,1,6,15,1030.0,0.0,14.0,...,1.0,0.0,0.0,1.0,1.0,0.0,2.0,3.0,1.0,0.610225
2,1,2023-01-15,6,1.0,1,6,15,1030.0,0.0,14.0,...,2.0,3.0,1.0,0.0,2.0,2.0,2.0,4.0,1.0,2.055764
3,1,2023-01-15,9,0.0,1,6,15,1030.0,0.0,14.0,...,1.0,1.0,1.0,0.0,2.0,0.0,2.0,4.0,1.0,1.146677
4,1,2023-01-15,12,8.0,1,6,15,1030.0,0.0,14.0,...,1.0,2.0,1.0,3.0,4.0,3.0,7.0,11.0,13.0,5.979035


In [21]:
print(f'validation dataframe shape : {validation_df.shape}')
validation_df.head()

validation dataframe shape : (29344, 33)


,Location,Date,Hour_interval,Demand,zone,day_of_week,day_of_month,group_max,group_min,max_previous_week_interval,...,previous_6day_interval,previous_week_interval,previous_8day_interval,previous_9day_interval,previous_10day_interval,previous_11day_interval,previous_12day_interval,previous_13day_interval,previous_2week_interval,Predicted_demand
496,1,2023-03-18,0,1.0,1,5,18,1030.0,0.0,9.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.354153
497,1,2023-03-18,3,2.0,1,5,18,1030.0,0.0,9.0,...,4.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,2.0,1.222646
498,1,2023-03-18,6,7.0,1,5,18,1030.0,0.0,9.0,...,1.0,0.0,2.0,3.0,1.0,1.0,1.0,4.0,1.0,0.574187
499,1,2023-03-18,9,2.0,1,5,18,1030.0,0.0,9.0,...,5.0,0.0,2.0,2.0,1.0,2.0,0.0,1.0,2.0,1.413134
500,1,2023-03-18,12,2.0,1,5,18,1030.0,0.0,9.0,...,5.0,5.0,1.0,1.0,8.0,2.0,2.0,3.0,0.0,4.600105


# Model Training

## **Gradient Boosting Regressor**

### Model Tuning

In [22]:
def grid_search(test_parameters, train_data, validation_data, feature_list):
    best_score = float('inf')
    best_params = None
    param_combination = product(*test_parameters.values())
    
    for param in param_combination:

        param_dict = dict(zip(test_parameters.keys(), param))
        model = RandomForestRegressor(**param_dict)
        model.fit(train_data[feature_list], train_data['Demand'])
        
        validation_predict_df = model.predict(validation_data[feature_list])
        score = mean_squared_error(validation_data['Demand'], validation_predict_df)
        
        if score<best_score:
            best_score = score
            best_params = param_dict
            
    return best_params, best_score

In [ ]:
if AUTO_TUNE:
    params_test = {
                    'oob_score': [True],
                    'max_features': [0.7, 0.8], 
                    'max_depth': [10, 12], 
                    'max_samples': [0.8, 0.9], 
                    'min_samples_leaf': [8, 10, 15],
                    'n_estimators': [1000, 1200]
                }

    best_params, best_score = grid_search(
        test_parameters = params_test,
        train_data = gridsearch_train_df,
        validation_data = validation_df,
        feature_list = FEATURE_LIST
        )
    
    print(best_params, best_score)
else:
    best_params = {
                    'oob_score': True,
                    'max_features': 0.7, 
                    'max_depth': 14, 
                    'max_samples': 0.8, 
                    'min_samples_leaf': 10,
                    'n_estimators': 1200
                   }


### Prediction

In [ ]:
def model_predict(model, train_data, test_data, feature_list):

    model.fit(train_data[feature_list], train_data['Demand'])
    
    train_predict_df  = model.predict(train_data[feature_list])
    test_predict_df  = model.predict(test_data[feature_list])

    return train_predict_df, test_predict_df

In [ ]:
model = RandomForestRegressor(**best_params)
train_prediction_df, test_prediction_df = model_predict(model, train_df, test_df, FEATURE_LIST)

### Visualization

In [ ]:
def prediction_visualization(train_data, test_data, train_prediction_df, test_prediction_df):

    predicted_train_df = train_data
    predicted_test_df = test_data
    predicted_train_df['Predicted'] = train_prediction_df
    predicted_test_df['Predicted'] = test_prediction_df
    
    train_data = train_data.groupby(['Date','Hour_interval'])['Demand'].sum()
    test_data = test_data.groupby(['Date','Hour_interval'])['Demand'].sum()
    predicted_train_df = predicted_train_df.groupby(['Date','Hour_interval'])['Predicted'].sum()
    predicted_test_df = predicted_test_df.groupby(['Date','Hour_interval'])['Predicted'].sum()

    plt.figure(figsize=(30,10))
    plt.title('Train', fontsize = 30)
    plt.plot(range(1,len(train_data)+1), train_data)
    plt.plot(range(1,len(train_data)+1), predicted_train_df)
    plt.xlabel('time interval', fontsize = 15)
    plt.legend(["Real Value", "Predicted"], loc ="lower right", fontsize = 15)
    plt.show()

    plt.figure(figsize=(30,10))
    plt.title('Test', fontsize = 30)
    plt.plot(range(1,len(test_data)+1), test_data)
    plt.plot(range(1,len(test_data)+1), predicted_test_df)
    plt.xlabel('time interval', fontsize = 15)
    plt.legend(["Real Value", "Predicted"], loc ="lower right", fontsize = 15)
    plt.show()

In [ ]:
prediction_visualization(train_df, test_df, train_prediction_df, test_prediction_df)

### Evaluation

In [ ]:
def evaluate(metric, metric_name, true_values, predicted_values):
    print(f'{metric_name} : {metric(true_values, predicted_values)}')

In [ ]:
def smape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred))
    smape = np.mean(numerator / denominator)
    return smape * 100

In [ ]:
def evaluation(model_name, train_df, test_df, train_prediction_df, test_prediction_df):
    print(f'{model_name} train scores:')


    evaluate(mean_absolute_error, 'MAE', train_df['Demand'], train_prediction_df)
    evaluate(mean_squared_error, 'MSE', train_df['Demand'], train_prediction_df)
    evaluate(mean_absolute_percentage_error, 'MAPE', train_df['Demand'], train_prediction_df)
    evaluate(smape, 'smape', train_df['Demand'], train_prediction_df)

    print(f'{model_name} test scores:')

    evaluate(mean_absolute_error, 'MAE', test_df['Demand'], test_prediction_df)
    evaluate(mean_squared_error, 'MSE', test_df['Demand'], test_prediction_df)
    evaluate(mean_absolute_percentage_error, 'MAPE', test_df['Demand'], test_prediction_df)
    evaluate(smape, 'smape', test_df['Demand'], test_prediction_df)


In [ ]:
evaluation('RF', train_df, test_df, train_prediction_df, test_prediction_df)

### Feature Importance and SHAPE

In [ ]:
importance = model.feature_importances_
sorted_indices = np.argsort(importance)[::-1]
plt.figure(figsize=(8, 6))
plt.barh(range(len(FEATURE_LIST)), importance[sorted_indices])
plt.yticks(range(len(FEATURE_LIST)), [FEATURE_LIST[i] for i in sorted_indices], rotation=45, ha='right')
plt.xlabel('Importance')
plt.ylabel('feature')
plt.title('Feature Importances in Random Forest Model')
plt.tight_layout()
plt.show()

# File Saving

In [ ]:
def save_predictions(dataset, path):
    dataset.to_parquet(path, index=False)

In [ ]:
def prediction_labeling(pred_df, labeled_df):
    labeled_df.reset_index(inplace = True)
    labeled_prediction_df = labeled_df[['Location', 'Date', 'Hour_interval']]
    labeled_prediction_df['Predicted_demand'] = pred_df
    return labeled_prediction_df

In [ ]:
labeled_prediction_df = prediction_labeling(test_prediction_df, test_df)

In [ ]:
print(f'labeled prediction dataframe shape : {labeled_prediction_df.shape}')
labeled_prediction_df.head()

In [ ]:
if add_ridge_feature:
    OUTPUT_PATH = RF_ridge_PATH
else:
    OUTPUT_PATH = RF_PATH

save_predictions(labeled_prediction_df, OUTPUT_PATH)